#Used Car Price Prediction
##1-> Problem Statement:
this dataset consists of used cars sold on cardekho.com in India as well as important features of these cars. If user can predict the price of the car based on input features. Prediction results can be used to give new seller the price suggestions based on market Conditions.

##2-> Data Collection
The dataset is collected from scapping from cardekho website. The data consista of 13 columns and 15411 rows

In [150]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [151]:
df=pd.read_csv('/content/cardekho_imputated.csv',index_col=[0])

In [152]:
df.sample(5)

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
9014,Maruti Alto,Maruti,Alto,6,22000,Individual,Petrol,Manual,22.74,796,47.30,5,249000
16233,Hyundai Santro,Hyundai,Santro,2,6932,Dealer,Petrol,Automatic,20.30,1086,68.07,5,650000
18727,Toyota Innova,Toyota,Innova,8,75000,Dealer,Diesel,Manual,12.80,2494,102.00,7,650000
11684,Maruti Swift,Maruti,Swift,5,60000,Individual,Petrol,Manual,20.40,1197,83.11,5,450000
14811,Hyundai Verna,Hyundai,Verna,8,120000,Individual,Petrol,Manual,17.43,1396,105.50,5,390000


#Cleaning the dataset

In [153]:
#Checking null values
df.isnull().sum()

,0
car_name,0
brand,0
model,0
vehicle_age,0
km_driven,0
seller_type,0
fuel_type,0
transmission_type,0
mileage,0
engine,0


In [154]:
#Remove unnecessary columns
df.drop('car_name',axis=1,inplace=True)
df.drop('brand',axis=1,inplace=True)

In [155]:
df.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [156]:
df['model'].unique()

array(['Alto', 'Grand', 'i20', 'Ecosport', 'Wagon R', 'i10', 'Venue',
       'Swift', 'Verna', 'Duster', 'Cooper', 'Ciaz', 'C-Class', 'Innova',
       'Baleno', 'Swift Dzire', 'Vento', 'Creta', 'City', 'Bolero',
       'Fortuner', 'KWID', 'Amaze', 'Santro', 'XUV500', 'KUV100', 'Ignis',
       'RediGO', 'Scorpio', 'Marazzo', 'Aspire', 'Figo', 'Vitara',
       'Tiago', 'Polo', 'Seltos', 'Celerio', 'GO', '5', 'CR-V',
       'Endeavour', 'KUV', 'Jazz', '3', 'A4', 'Tigor', 'Ertiga', 'Safari',
       'Thar', 'Hexa', 'Rover', 'Eeco', 'A6', 'E-Class', 'Q7', 'Z4', '6',
       'XF', 'X5', 'Hector', 'Civic', 'D-Max', 'Cayenne', 'X1', 'Rapid',
       'Freestyle', 'Superb', 'Nexon', 'XUV300', 'Dzire VXI', 'S90',
       'WR-V', 'XL6', 'Triber', 'ES', 'Wrangler', 'Camry', 'Elantra',
       'Yaris', 'GL-Class', '7', 'S-Presso', 'Dzire LXI', 'Aura', 'XC',
       'Ghibli', 'Continental', 'CR', 'Kicks', 'S-Class', 'Tucson',
       'Harrier', 'X3', 'Octavia', 'Compass', 'CLS', 'redi-GO', 'Glanza',
       

In [157]:
## Getting all different types of features
num_features=[feature for feature in df.columns if df[feature].dtype!='O']
print('Numerical features:',len(num_features))
cat_features=[feature for feature in df.columns if df[feature].dtype=='O']
print('Categorical features:',len(cat_features))
discrete_features=[feature for feature in num_features if len(df[feature].unique())<25]
print('Discrete features:',len(discrete_features))
continuous_features=[feature for feature in num_features if feature not in discrete_features]
print('Continuous features:',len(continuous_features))

Numerical features: 7
Categorical features: 4
Discrete features: 2
Continuous features: 5


In [158]:
##Independent and Dependent Features
from sklearn.model_selection import train_test_split
X=df.drop('selling_price',axis=1)
y=df['selling_price']

In [159]:
X.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


#Feature Encoding and Scaling

In [160]:
df['model'].value_counts()

,count
model,
i20,906
Swift Dzire,890
Swift,781
Alto,778
City,757
...,...
Altroz,1
C,1
Ghost,1


In [161]:
# lets assign labels
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X['model']=le.fit_transform(X['model'])

In [162]:
X.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,7,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,54,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,118,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,7,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,38,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


In [163]:
len(X['seller_type'].unique()),len(X['fuel_type'].unique()),len(X['transmission_type'].unique())

(3, 5, 2)

In [164]:
# Create Column Transformer with 3 types of transformer
num_features=X.select_dtypes(exclude='object').columns
onehot_columns=['seller_type','fuel_type','transmission_type']

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer=StandardScaler()
oh_transformer=OneHotEncoder()

preprocessor=ColumnTransformer(
    [
     ('OneHotEncoder',oh_transformer,onehot_columns),
     ('StandardScaler',numeric_transformer,num_features)
    ],remainder='passthrough'
)

In [165]:
X=preprocessor.fit_transform(X)

In [166]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,-1.519714,0.983562,1.247335,-0.000276,-1.324259,-1.263352,-0.403022
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,-0.225693,-0.343933,-0.690016,-0.192071,-0.554718,-0.432571,-0.403022
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.536377,1.647309,0.084924,-0.647583,-0.554718,-0.479113,-0.403022
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,-1.519714,0.983562,-0.360667,0.292211,-0.936610,-0.779312,-0.403022
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,-0.666211,-0.012060,-0.496281,0.735736,0.022918,-0.046502,-0.403022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15406,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.508844,0.983562,-0.869744,0.026096,-0.767733,-0.757204,-0.403022
15407,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,-0.556082,-1.339555,-0.728763,-0.527711,-0.216964,-0.220803,2.073444
15408,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.407551,-0.012060,0.220539,0.344954,0.022918,0.068225,-0.403022
15409,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.426247,-0.343933,72.541850,-0.887326,1.329794,0.917158,2.073444


In [167]:
X

array([[ 0.        ,  1.        ,  0.        , ..., -1.32425883,
        -1.26335238, -0.40302241],
       [ 0.        ,  1.        ,  0.        , ..., -0.55471774,
        -0.43257082, -0.40302241],
       [ 0.        ,  1.        ,  0.        , ..., -0.55471774,
        -0.47911321, -0.40302241],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.02291783,
         0.06822523, -0.40302241],
       [ 1.        ,  0.        ,  0.        , ...,  1.32979434,
         0.91715831,  2.07344426],
       [ 1.        ,  0.        ,  0.        , ...,  0.02099878,
         0.39588361, -0.40302241]])

In [168]:
# separate dataset into train and test
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape,X_test.shape

((12328, 17), (3083, 17))

#Model Training and Model Selection

In [169]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [170]:
#Function to evaluate model
def evaluate_model(true,predicted):
  mae=mean_absolute_error(true,predicted)
  mse=mean_squared_error(true,predicted)
  rmse=np.sqrt(mse)
  r2=r2_score(true,predicted)
  return mae,rmse,r2

In [171]:
#beginning model training
models={
    "Linear Regression": LinearRegression(),
    "Lasso":Lasso(),
    "Ridge":Ridge(),
    "K-Neighbors Regressor":KNeighborsRegressor(),
    "Decision Tree":DecisionTreeRegressor(),
    "Random Forest Regressor":RandomForestRegressor()
}

for i in range(len(list(models))):
  model=list(models.values())[i]
  model.fit(X_train,y_train)

  #Make predictions
  y_train_pred=model.predict(X_train)
  y_test_pred=model.predict(X_test)

  #Evaluate train and test data
  model_train_mae,model_train_rmse,model_train_r2=evaluate_model(y_train,y_train_pred)
  model_test_mae,model_test_rmse,model_test_r2=evaluate_model(y_test,y_test_pred)

  print(list(models.keys())[i])
  print('Model performance for training set')
  print('-RMSE:{:.4f}'.format(model_train_rmse))
  print('-MAE:{:.4f}'.format(model_train_mae))
  print('-R2:{:.4f}'.format(model_train_r2))
  print('-'*30)
  print('Model performance for test set')
  print('-RMSE:{:.4f}'.format(model_test_rmse))
  print('-MAE:{:.4f}'.format(model_test_mae))
  print('-R2:{:.4f}'.format(model_test_r2))
  print('='*30)
  print('\n')

Linear Regression
Model performance for training set
-RMSE:553855.6665
-MAE:268101.6071
-R2:0.6218
------------------------------
Model performance for test set
-RMSE:502543.5930
-MAE:279618.5794
-R2:0.6645


Lasso
Model performance for training set
-RMSE:553855.6708
-MAE:268099.3639
-R2:0.6218
------------------------------
Model performance for test set
-RMSE:502542.6393
-MAE:279614.9121
-R2:0.6645


Ridge
Model performance for training set
-RMSE:553856.3122
-MAE:268060.0183
-R2:0.6218
------------------------------
Model performance for test set
-RMSE:502533.3688
-MAE:279558.1057
-R2:0.6645


K-Neighbors Regressor
Model performance for training set
-RMSE:327375.9476
-MAE:92307.5762
-R2:0.8679
------------------------------
Model performance for test set
-RMSE:255100.0858
-MAE:113683.7901
-R2:0.9136


Decision Tree
Model performance for training set
-RMSE:20797.2352
-MAE:5164.8199
-R2:0.9995
------------------------------
Model performance for test set
-RMSE:300808.3744
-MAE:124315.9

## As we can see KNN and Random Forest Classifier, these are performing really good based on r2 score. So lets perform hyperparameter tuning

In [172]:
## Initializing params
knn_params={"n_neighbors":[2,3,10,20,40,50]}
rf_params={
    "max_depth":[5,8,15,25,30,None],
    "min_samples_leaf":[1,2,5,10,15,100],
    "min_samples_split":[2,5,10,15,100],
    "n_estimators":[100,200,300,400,500]
    }

In [173]:
#Model lists for hyperparameter tuning
randomcv_models=[('KNN',KNeighborsRegressor(),knn_params),('Random Forest',RandomForestRegressor(),rf_params)]

In [174]:
from sklearn.model_selection import RandomizedSearchCV
model_param={}
for name,model,params in randomcv_models:
  random=RandomizedSearchCV(estimator=model,param_distributions=params,n_iter=100,cv=3,verbose=2,n_jobs=-1)
  random.fit(X_train,y_train)
  model_param[name]=random.best_params_

  for model_name in model_param:
    print(f"The best parameters for {model_name} are {model_param[model_name]}")

Fitting 3 folds for each of 6 candidates, totalling 18 fits
The best parameters for KNN are {'n_neighbors': 10}
Fitting 3 folds for each of 100 candidates, totalling 300 fits
The best parameters for KNN are {'n_neighbors': 10}
The best parameters for Random Forest are {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 15}


In [175]:
#Retraining the model with parameters obtained for KNN and RandomForestRegressor
models={
    "K-Neighbors Regressor":KNeighborsRegressor(n_neighbors=10),
    "Random Forest Regressor":RandomForestRegressor(n_estimators=100,min_samples_split=2,min_samples_leaf=1,max_depth=15)
}

for i in range(len(list(models))):
  model=list(models.values())[i]
  model.fit(X_train,y_train)

  #Make predictions
  y_train_pred=model.predict(X_train)
  y_test_pred=model.predict(X_test)

  #Evaluate train and test data
  model_train_mae,model_train_rmse,model_train_r2=evaluate_model(y_train,y_train_pred)
  model_test_mae,model_test_rmse,model_test_r2=evaluate_model(y_test,y_test_pred)

  print(list(models.keys())[i])
  print('Model performance for training set')
  print('-RMSE:{:.4f}'.format(model_train_rmse))
  print('-MAE:{:.4f}'.format(model_train_mae))
  print('-R2:{:.4f}'.format(model_train_r2))
  print('-'*30)
  print('Model performance for test set')
  print('-RMSE:{:.4f}'.format(model_test_rmse))
  print('-MAE:{:.4f}'.format(model_test_mae))
  print('-R2:{:.4f}'.format(model_test_r2))
  print('='*30)
  print('\n')

K-Neighbors Regressor
Model performance for training set
-RMSE:365913.6901
-MAE:104814.0493
-R2:0.8349
------------------------------
Model performance for test set
-RMSE:267334.9500
-MAE:119023.4269
-R2:0.9051


Random Forest Regressor
Model performance for training set
-RMSE:129840.2783
-MAE:52820.9855
-R2:0.9792
------------------------------
Model performance for test set
-RMSE:224813.8292
-MAE:100606.1743
-R2:0.9329


